In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:24pt;}
div.text_cell_render.rendered_html{font-size:20pt;}
div.text_cell_render li, div.text_cell_render p, code{font-size:22pt; line-height:40px;}
div.output {font-size:24pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:24pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:24pt;padding:5px;}
table.dataframe{font-size:24px;}
</style>
"""))

# <span style="color:rgb(0,200,100);">1. 패키지 </span>

In [5]:
# python-dotenv, langchain(1.2.0),  langchain-openai,  langchain-pinecone,
# pandas,  langchain-community, docx2txt,  langchain-text_splitters,  langchain_ollama

# <span style="color:rgb(0,200,100);">2. 환경설정(환경변수, 시스템파라미터변수) </span>

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_LLM_MODEL = "gpt-4o-mini"
OPENAI_EMBEDDING_MODEL = "text-embedding-3-large" # 차원수 3072

PINECONE_INDEX_NAME = "better-rag-index"
PINECONE_INDEX_DEMENSION = 3072
PINECONE_INDEX_METRIC = "cosine"
PINECONE_INDEX_REGION = "us-east-1"
PINECONE_INDEX_CLOUD = "aws"

# <span style="color:rgb(0,200,100);">3. 문서를 chunk로 분할하기 </span>

In [7]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
loader = Docx2txtLoader('./data/소득세법_with_markdown.docx')
document = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200, 
    separators=["\n\n", "\n", " ", ""]
)
# documents = loader.load_and_split(text_splitter)
documents = text_splitter.split_documents(document)
print(f"총 {len(documents)}개 청크 생성")

총 194개 청크 생성


# <span style="color:rgb(0,200,100);">4. metadata 추가하기 </span>
- 청크 내용의 카테고리, 청크내용의 title, 조항

In [13]:
import re
def remove_special_chars(text:str) -> str:
    "특수문자 및 \n제거(:는 그대로)"
    # \n제거
    text = text.replace("\n", " ")
    # 한글, 영문, 숫자, 공백, 마침표, 콤마, :만 남기고 전부 제거
    cleaned = re.sub(r'[^가-힣a-zA-Z0-9\s,\.:]', '', text)
    # 불필요한 중복 공백을 제거
    cleaned = re.sub(r'\s+', ' ', cleaned).strip()
    return cleaned
# 사용예시
content = "**소득세 납세 의무 **: \n\n이 법 또는 「법인세법」에 따라 소득에 대한 소득세를 부과"
print(remove_special_chars(content))

소득세 납세 의무 : 이 법 또는 법인세법에 따라 소득에 대한 소득세를 부과


In [ ]:
# 제목을 추출하는 함수(exaone3.5) : powershell이나 cmd창에서 ollama pull exaone3.5:2.4b
from langchain_ollama import ChatOllama
def extract_title_with_llm(content):
    "exaone3.5를 사용하여 content의 제목을 추출"